In [194]:
from dask.distributed import Client
client = Client('dask-scheduler:8786')
client

<Client: 'tcp://172.18.0.2:8786' processes=6 threads=6, memory=12.85 GiB>

# Install skimage and dask image

In [23]:
import os
os.system('conda install -c anaconda scikit-image -y')

def skimage_install():
    out=os.system('conda install -c anaconda scikit-image -y')
    return out

client.run(skimage_install)

{'tcp://172.18.0.4:37723': 0,
 'tcp://172.18.0.5:46203': 0,
 'tcp://172.18.0.6:43397': 0,
 'tcp://172.18.0.7:44457': 0,
 'tcp://172.18.0.8:40843': 0,
 'tcp://172.18.0.9:34639': 0}

In [25]:
os.system('conda install -c conda-forge dask-image -y')

def skimage_install():
    out=os.system('conda install -c conda-forge dask-image -y')
    return out

client.run(skimage_install)

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - dask-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py39hf3d152e_0         155 KB  conda-forge
    conda-4.13.0               |   py39hf3d152e_1         998 KB  conda-forge
    dask-image-2021.12.0       |     pyhd8ed1ab_0          47 KB  conda-forge
    pims-0.6.1                 |     pyhd8ed1ab_0          66 KB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    slicerator-1.1.0           |     pyhd8ed1ab_0          13 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INS

{'tcp://172.18.0.4:37723': 0,
 'tcp://172.18.0.5:46203': 0,
 'tcp://172.18.0.6:43397': 0,
 'tcp://172.18.0.7:44457': 0,
 'tcp://172.18.0.8:40843': 0,
 'tcp://172.18.0.9:34639': 0}

In [199]:
import numpy as np
import dask_image
import dask.array as da
import dask.dataframe as dd
import dask.delayed
from dask.array.image import imread
filename_pattern = os.path.join('temp-tiles', 'image-*.png')

def load_turtles(path, kind='train'):
    f  = os.path.join(path, 'face/face*.jpg')
    nf = os.path.join(path, 'noface/noface*.jpg')

    #load all images
    images = da.concatenate([imread(f), imread(nf)])
        
    ##create numpy list of ,matching labels
    face_lb = np.ones(len(os.listdir(os.path.join(path, 'face')))) #fetch the 
    no_face_lb= np.negative(np.ones(len(os.listdir(os.path.join(path, 'noface')))))
    labels = np.append(face_lb, no_face_lb)
    labels= da.from_array(labels)
    return images, labels 

data_path = '/opt/workspace/data/'
X, y = load_turtles(data_path)
y

dask.array<array, shape=(4000,), dtype=float64, chunksize=(4000,), chunktype=numpy.ndarray>

## Just testing doing the preprocessing straight away before splitting

We actually are reducing the dimensionality of the array as we convert the images into 2D from RGB array. 

In [200]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
from skimage.feature import hog
import skimage

class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
     
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:

            return np.array([local_hog(img) for img in X])

In [201]:
@dask.delayed
def SGD_preprocessing_d(X):
    # create an instance of each transformer
    grayify = RGB2GrayTransformer()
    hogify = HogTransformer(
        pixels_per_cell=(14, 14), 
        cells_per_block=(2,2), 
        orientations=9, 
        block_norm='L2-Hys'
    )
    scalify = StandardScaler()
    # call fit_transform on each transform converting X_train step by step
    X_gray = grayify.fit_transform(X)
    X_hog = hogify.fit_transform(X_gray)
    X_prepared = scalify.fit_transform(X_hog)
    #return np.shape(X_prepared)
    return X_prepared

chunk_size = 500
X=X.rechunk(chunks= (chunk_size, 198,198, 3)).to_delayed().ravel()

#X_prepared = SGD_preprocessing_d(X)
#dd.compute(*[SGD_preprocessing_d(i) for i in X])
#dd.compute(*[da.from_delayed(SGD_preprocessing_d(i), (1,6084), dtype=np.float64) for i in X])
X=da.concatenate([da.from_delayed(SGD_preprocessing_d(i), (chunk_size, 6084), dtype=np.float64) for i in X])

In [202]:
from IPython.display import display
permutation = da.random.permutation(len(X))

X = X[permutation]
y = y[permutation]

m_train = int(len(X)*0.6) 
m_test = len(X) - m_train

X_train, X_test = X[:m_train], X[m_train:]
y_train, y_test = y[:m_train], y[m_train:]
print(m_train, m_test, m_train+m_test)

labels, freqs = np.unique(y_train, return_counts=True)
print("Labels in training dataset: ", labels)
print("Frequencies in training dataset: ", freqs)
display(X_train, y_train)

2400 1600 4000
Labels in training dataset:  dask.array<getitem, shape=(nan,), dtype=float64, chunksize=(nan,), chunktype=numpy.ndarray>
Frequencies in training dataset:  dask.array<getitem, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>


dask.array<getitem, shape=(2400, 6084), dtype=float64, chunksize=(2400, 6084), chunktype=numpy.ndarray>

dask.array<getitem, shape=(2400,), dtype=float64, chunksize=(2400,), chunktype=numpy.ndarray>

In [203]:
from dask_ml.datasets import make_classification
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

X_train_ugly = X_train.rechunk(chunks= (400, 6084))
y_train_ugly = y_train.rechunk(chunks= (400))
X_train_ugly = X_train_ugly.compute()
y_train_ugly = y_train_ugly.compute()

# display(X_train_ugly, y_train_ugly)
# estimator = SGDClassifier(random_state=42, max_iter=100)
# clf = Incremental(estimator)
# clf.fit(X_train_ugly, y_train_ugly, classes=["face", "no face"])
# Incremental(estimator=SGDClassifier(random_state=42, max_iter=1000))

In [204]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train_ugly, y_train_ugly)

SGDClassifier(random_state=42)

In [205]:
y_pred = sgd_clf.predict(X_train_ugly)
print('Percentage correct: ', 100*np.sum(y_pred == y_train_ugly)/len(y_train_ugly))

Percentage correct:  96.70833333333333
